In [4]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import pandas as pd
import time

In [5]:
URL = 'https://comic.naver.com/webtoon/weekday.nhn'
html = requests.get(URL).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
soup = BeautifulSoup(html, 'html.parser')

title = soup.find_all('a', {'class' : 'title'}) # a태그에서 class='title'인 html소스를 찾아 할당
id_list = []
title_list = []
author_list = []
day_list = []
genre_list = []
story_list = []
score_list = []
platform_list = []

num = 0

driver = webdriver.Chrome('C:/chromedriver.exe') # 크롬 사용하니까
driver.get(URL)

for i in range(len(title)):
    sleep(0.5) # 크롤링 중간 중간 텀을 주어 과부하 생기지 않도록

    page = driver.find_elements_by_class_name('title')
    page[i].click() #월요일 첫 번째 웹툰부터 순서대로 클릭

    sleep(0.5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') # 이동한 페이지 주소 읽고 파싱

    day = soup.find_all('ul', {'class' : 'category_tab'})
    day = day[0].find('li', {'class' : 'on'}).text[0:1] # 요일 수집
    
    t = title[i].text
    if (t in title_list):  # 요일 두 개 이상이면 요일만 추가함
        day_list[title_list.index(t)] += ', ' + day
        driver.back()
        continue

    id_list.append(num) ; num += 1  # id 리스트에 추가
    title_list.append(t)  # 제목 리스트에 추가
    day_list.append(day) # 요일 리스트에 추가
    platform_list.append('네이버 웹툰') # 플랫폼 리스트에 추가

    author = soup.find_all('h2') # 두 번째 h2태그에 있음
    author = author[1].find('span', {'class' : 'wrt_nm'}).text[8:] # 7칸의 공백 후 8번 째부터 작가 이름임
    author_list.append(author) # 작가 리스트에 추가

    genre = soup.find('span', {'class' : 'genre'}).text # 장르 수집
    genre_list.append(genre) # 장르 리스트에 추가

    story = soup.find_all('p') # 줄거리 수집
    story = str(story[3])
    story = story.replace('<p>', '').replace('</p>', '').replace('<br/>', '\n') # <br>을 개행으로 바꾸기
    story_list.append(story) # 줄거리 리스트에 추가
    
    #최신 별점 평균 점수 수집 (최대 10화 분량)
    score = soup.find_all('strong')
    scorelist=[] ; ii=9
    while score[ii].text[0].isnumeric()==True:
        scorelist.append(float(score[ii].text))
        ii +=1
    score_list.append(sum(scorelist)/len(scorelist))
    
    time.sleep(0.5)

    driver.back() # 뒤로 가기

############################################크롤링 끝############################################

total_data = pd.DataFrame()
total_data['id'] = id_list
total_data['title'] = title_list
total_data['author'] = author_list
total_data['day'] = day_list
total_data['genre'] = genre_list
total_data['story'] = story_list
total_data['score'] = score_list
total_data['platform'] = platform_list
total_data.to_csv('네이버_웹툰.csv', encoding='utf-8-sig')

KeyboardInterrupt: 

In [16]:
df = pd.read_csv('네이버_웹툰.csv', encoding='utf-8-sig')

In [25]:
import re

def preprocess_sentence_kr(w):
  w = w.strip()
  w = re.sub(r"[^0-9가-힣?.!,¿]+", " ", w) # \n도 공백으로 대체해줌
  w = w.strip()
  return w

In [26]:
df['story'] = [preprocess_sentence_kr(l) for l in df['story']]

In [27]:
df

,Unnamed: 0,id,title,author,day,genre,story,score,platform
0,0,0,참교육,채용택 / 한가람,월,"스토리, 액션",무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다! 부활남 채용...,9.776,네이버 웹툰
1,1,1,신의 탑,SIU,월,"스토리, 판타지",자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년 그리고 그런 소년을 시험하는 탑,9.499,네이버 웹툰
2,2,2,뷰티풀 군바리,설이 / 윤성원,월,"스토리, 드라마",여자도 군대에 간다면? 본격 여자도 군대 가는 만화!,9.660,네이버 웹툰
3,3,3,윈드브레이커,조용석,월,"스토리, 스포츠",혼자서 자전거를 즐겨타던 모범생 조자현. 원치 않게 자전거 크루의 일에 자꾸 휘말리...,8.795,네이버 웹툰
4,4,4,팔이피플,매미 / 희세,월,"스토리, 드라마","마스크걸 , 위대한 방옥숙 매미 희세 작가의 신작! 에서 육아용품 파는 평범한 유부...",9.880,네이버 웹툰
...,...,...,...,...,...,...,...,...,...
440,440,440,제타,하지,일,"스토리, 스릴러",극심한 슬럼프에 안드로이드 제타 의 그림을 표절해버린 금세기 최고의 천재 화가 최후...,9.760,네이버 웹툰
441,441,441,푸른불꽃,닺,일,"스토리, 드라마",유년시절 트라우마를 겪은 문가야 . 새롭게 전학 온 학교에서 완벽한 모범생이자 모두...,9.840,네이버 웹툰
442,442,442,데이즈,양수,일,"스토리, 스릴러",대한민국 최고 재벌가의 첫째 아들이자 집안의 망나니 한지한 은 오늘도 거하게 취했다...,8.705,네이버 웹툰
443,443,443,샤인 스타,김현,일,"스토리, 로맨스",전학생 선유별이 좋아하는 김샤인은 학교에서 유명한 달리기 스타. 잘생긴 얼굴에 뛰어...,9.951,네이버 웹툰


In [28]:
df.to_csv('NAVER.csv', encoding='euc-kr')